# Option Discovery

## Introduction

Using hierarchies of temporally extended actions can dramatically reduce learning time. In 1999, [Sutton, Precup and Singh][@Sutton1999] proposed a kind of temporally extended actions, the option framework. An option $ o $ is a triple of $ < I_o, \pi_o, \beta_o > $; $ I_o $ is an initiation set of states, $ \pi_o $ is an encapsulated policy and $ \beta_o $ a termination condidation.[Dietterich][@Dietterich2000] mentioned in his paper the difficulty of Hierarchical reinforcement learning is how to discover its structure. To uses previous knowledge to design the options manually costs much and can not work well for dynamic environments. The approaches of automated discovery of options has been proposed. In this report, I will give some introduction and comparison of them.



[@Sutton1999]:http://www-anw.cs.umass.edu/~barto/courses/cs687/Sutton-Precup-Singh-AIJ99.pdf
[@Dietterich2000]:http://www.jair.org/media/639/live-639-1834-jair.pdf

## Diverse Density from [McGovern and Barto][@McGovern]
The diverse density was proposed by [Maron][@Maron1998].
\begin{equation}
DD(t) = Pr(t|B_1^+, …, B_n^+, B_1^-, …, B_m^-) \\
= \prod_{1 \leq i \leq n} Pr(t|B_i^+)  \prod_{1 \leq i \leq m} Pr(t|B_i^-)
\end{equation}
$ B_i^+ $ is the $ i_th $ positive bag, while $ B_i^- $ is the $ i_th $ negative bag.
When the agent is passing through the successful trajectories, the region with maximum diverse density will be a bottleneck region.This is the pseudo code of the option-discovery algorithm.
``` python
init an empty array to store successful trajectories
for each trial:
    interact with environment/Learn using RL
    add observed successful trajectory to array
    create positive or negative bag from its state trajectory
    search for diverse density peaks
    for each peak concept c found:
        update the running average by c = lambda * (c + 1 )
        if c is above threshold:
            if c passes the static filter
                Create a new option of reaching concept c
            init I by examining trajectory array
            set Beta(c) = 1, Beta(S-I) = 1, Beta() = 0 else
            init policy pi using experience replay
```
### Experiments and Results
McGovern evaluated the diverse density method on a Two-Room Gridworld enviroment. The start state is in the left room, while the goal is in the right room.
<img src="files/tworoom.png" width="480">
After 25 trials, the following figure from [McGovern and Barto][@McGovern] shows the average log likelihood of the diverse density for stats.From this figure, the region with most light is also the bottleneck of this environment.
<img src="files/tworoomDD.png" width="400">
[McGovern and Barto][@McGovern] shows that after discoverying option properly the learning is accelerated.
<img src="files/tworoomRL.png" width="400">
[@McGovern]:https://pdfs.semanticscholar.org/7eca/3acd1a4239d8a299478885c7c0548f3900a8.pdf

## Q-Cut from [Menache, Mannor and Shimkin][@Menache]
[Menache et al.][@Menache] got the inspiration form the graph theory. They presented the Q-Cut algorithm to learn options from a dynamic environment. As the diverse density approach, this method also identifies bottlenecks.The idea is to perform the cut between the source and target nodes and estimate the quality of this cut. If it is a 'good' cut, a new option is found. This is the pseudo code of its algorithm.
``` python
init an empty array to store successful trajectories
for each trial:
    interact with environment/Learn using RL
    save state transition history
    if activating cut conditions are met, choose s,t from S:
        translate state transition history to a graph representation
        find a Minimum Cut partition [Ns, Nt] between nodes s and t
        if the quality is good:
            learn the option for reaching new derived bottlenecks
            for each state in Ns:
                using Experience-Replay
```
### Experiments and Results
[Menache et al.][@Menache] evaluated the Q-Cut approach on two different environments; one of the is Two-Room Gridworld same as [McGovern and Barto][@McGovern] did, the other one is 6-Room Gridworld. They show their result based on 50 trials. The steps to goal of the Q-Cut decline tremendous comparing to Q-Learning without using options after a few episodes.
<img src="files/tworoomQCUT.png" width="400">
Because [McGovern and Barto][@McGovern] also tested their method on this environment, it is interesting that [Menache et al.][@Menache] performs much better after a few episodes.

[@Menache]:http://webee.technion.ac.il/people/shimkin/PAPERS/QCut.pdf
[@McGovern]:https://pdfs.semanticscholar.org/7eca/3acd1a4239d8a299478885c7c0548f3900a8.pdf

## Betweenness from [Simsek and Barto][@Simsek]
In 2009, Simsek and Barto proposed another approach based on a graphical representation. They use betweenness which is a measure of centrality in a graph to identify a set of options. The betweenness is defined as follows,
\begin{equation}
g(v) = \sum_{s \neq v \neq t}\frac{\sigma_{st}(v)}{\sigma_{st}}
\end{equation}
where $ \sigma_{st} $ is the number of shortest paths from $ s $ to $ t $.
This is an example from [Simsek and Barto][@Simsek] showing the betweenness centrality of nodes.In both of them, the states with local maxima of betweenness are their candidate options.
<img src="files/example.png" width="500">
### Experiments and Results
[Simsek and Barto][@Simsek] evaluated their method in three different environments - Taxi, Playroom and  Tic-Tac-Toe.Here is the performence based on the same Two-Room Gridworld environment.
<img src="files/tworoomBW.png" width="400">
[@Simsek]:https://papers.nips.cc/paper/3411-skill-characterization-based-on-betweenness.pdf

## Spatio-Temporal Clustering from [Lakshminarayanan, Krishnamurthy, Kumar and Ravindran][@Lakshminarayanan]
[Lakshminarayanan et al.][@Lakshminarayanan] introduced Spatio-Temporal Clustering method to identify options in 2016. [PCCA+][@Weber] is a spectral clustering algorithm. The following is the pseudo code that they perform Option Discovery using SpatioTemporal Clustering.
``` python
while Not Converged do:
    interact with environment/Learn using RL
    save state transition history
    estimate model from sample trajectories
    operate PCCA+ on estimated model to derive abstract states and memberships
    discover options from the abstract states and memberships
    augment agent with new options
    update value functions and behavioral policy using
    SMDP Q Learning
```
### Experiments and Results
[Lakshminarayanan et al.][@Lakshminarayanan] evaluated their method on the environments of 3-Room Gridworld and Seaquest.
<img src="files/tworoomSP.png" width="500">
From this graph (left) learning without the reward structure, their method identified 3 separated abstract states; while the graph (right) learning with the reward structure, there are 4 abstract states which make the room 3 different.
[@Lakshminarayanan]:https://arxiv.org/pdf/1605.05359.pdf
[@Weber]:http://bioinformatics.rutgers.edu/Static/Publications/ZR-04-39.pdf

## Future Work
The above approaches have had varying degrees of success. In th future, it will be helpful to use some unified environments to evaluate them and make clear the advantages and disadvantages of them.

## Reference

1. Sutton, R. S., Precup, D., & Singh, S. (1999). Between MDPs and semi-MDPs: A framework for temporal abstraction in reinforcement learning. Artificial intelligence, 112(1-2), 181-211.
2. Dietterich, T. G. (2000). Hierarchical reinforcement learning with the MAXQ value function decomposition. J. Artif. Intell. Res.(JAIR), 13, 227-303.
3. McGovern, A., & Barto, A. G. (2001). Automatic discovery of subgoals in reinforcement learning using diverse density.
4. Menache, I., Mannor, S., & Shimkin, N. (2002, August). Q-cut—dynamic discovery of sub-goals in reinforcement learning. In European Conference on Machine Learning (pp. 295-306). Springer Berlin Heidelberg.
5. Şimşek, Ö., & Barto, A. G. (2009). Skill characterization based on betweenness. In Advances in neural information processing systems (pp. 1497-1504).
6. Lakshminarayanan, A. S., Krishnamurthy, R., Kumar, P., & Ravindran, B. (2016). Option Discovery in Hierarchical Reinforcement Learning using Spatio-Temporal Clustering. arXiv preprint arXiv:1605.05359.
7. Weber, M., Rungsarityotin, W., & Schliep, A. (2004). Perron cluster analysis and its connection to graph partitioning for noisy data. Konrad-Zuse-Zentrum für Informationstechnik Berlin.